# Binary encoding

In [66]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, DepthwiseConv2D
import tensorflow as tf

In [67]:
# Diccionarios para decodificación en función de índices
layer_type_options = {
    0: 'Conv2D', 
    1: 'BatchNorm', 
    2: 'MaxPooling', 
    3: 'Dropout', 
    4: 'Dense', 
    5: 'Flatten',
    6: 'DepthwiseConv2D',  
    7: 'DontCare',  
    8: 'Repetition'
}
filter_options = {3: 32, 2: 30, 1: 16, 0: 8}
stride_options = {0: 1, 1: 2}
dropout_options = {0: 0.2, 1: 0.3, 2: 0.4, 3: 0.5}
neuron_options = {3: 256, 2: 128, 1: 32, 0: 1}
activation_options = {0: 'relu', 1: 'leaky_relu', 2: 'sigmoid', 3: 'tanh'}

def encode_layer_params(layer_type_idx, param1=0, param2=0, param3=0):
    """
    Codifica una capa en formato de valores reales en función de los índices proporcionados.
    
    layer_type_idx : int : índice del tipo de capa según layer_type_options
    param1         : int : índice del primer parámetro (e.g., filtros, stride, etc.)
    param2         : int : índice del segundo parámetro (e.g., stride, dropout, etc.)
    param3         : int : índice del tercer parámetro (e.g., activación, etc.)
    """
    # Codificar la capa en base a su tipo e índices de parámetros
    return [layer_type_idx, param1, param2, param3]

def decode_layer_params(encoded_params):
    """
    Decodifica una capa en valores interpretables.
    
    encoded_params : list : [tipo de capa, param1, param2, param3]
    """
    layer_type_idx = encoded_params[0]
    layer_type = layer_type_options.get(layer_type_idx, 'DontCare')
    
    # Decodificar en función del tipo de capa
    if layer_type == 'Conv2D' or layer_type == 'DepthwiseConv2D':
        filters = filter_options.get(encoded_params[1], 8.0)
        strides = stride_options.get(encoded_params[2], 1.0)
        activation = activation_options.get(encoded_params[3], 'relu')
        return {
            'type': layer_type,
            'filters': filters,
            'strides': strides,
            'activation': activation
        }
    elif layer_type == 'BatchNorm':
        return {'type': 'BatchNorm'}
    elif layer_type == 'MaxPooling':
        strides = stride_options.get(encoded_params[1], 1.0)
        return {'type': 'MaxPooling', 'strides': strides}
    elif layer_type == 'Dropout':
        rate = dropout_options.get(encoded_params[1], 0.2)
        return {'type': 'Dropout', 'rate': rate}
    elif layer_type == 'Dense':
        neurons = neuron_options.get(encoded_params[1], 1.0)
        activation = activation_options.get(encoded_params[2], 'relu')
        return {'type': 'Dense', 'units': neurons, 'activation': activation}
    elif layer_type == 'Flatten':
        return {'type': 'Flatten'}
    elif layer_type == 'Repetition':
        return {
            'type': 'Repetition',
            'repetition_layers': int(encoded_params[1]),
            'repetition_count': int(encoded_params[2])
        }
    elif layer_type == 'DontCare':
        return {'type': "DontCare"}

    return None

# Ejemplos de codificación y decodificación
encoded_conv2d = encode_layer_params(0, 0, 0, 0)  # Conv2D con 8 filtros, 1 stride y activación ReLU
decoded_conv2d = decode_layer_params(encoded_conv2d)
print(f"\nCodificación real de Conv2D: {encoded_conv2d}")
print(f"Decodificación Conv2D: {decoded_conv2d}")

encoded_dropout = encode_layer_params(3, 1)  # Dropout con tasa de 0.3
decoded_dropout = decode_layer_params(encoded_dropout)
print(f"\nCodificación real de Dropout: {encoded_dropout}")
print(f"Decodificación Dropout: {decoded_dropout}")

encoded_dense = encode_layer_params(4, 2, 0)  # Dense con 128 neuronas y activación ReLU
decoded_dense = decode_layer_params(encoded_dense)
print(f"\nCodificación real de Dense: {encoded_dense}")
print(f"Decodificación Dense: {decoded_dense}")

encoded_repetition = encode_layer_params(8, 3, 5)  # Repetition para repetir las últimas 3 capas 5 veces
decoded_repetition = decode_layer_params(encoded_repetition)
print(f"\nCodificación real de Repetition: {encoded_repetition}")
print(f"Decodificación Repetition: {decoded_repetition}")



Codificación real de Conv2D: [0, 0, 0, 0]
Decodificación Conv2D: {'type': 'Conv2D', 'filters': 8, 'strides': 1, 'activation': 'relu'}

Codificación real de Dropout: [3, 1, 0, 0]
Decodificación Dropout: {'type': 'Dropout', 'rate': 0.3}

Codificación real de Dense: [4, 2, 0, 0]
Decodificación Dense: {'type': 'Dense', 'units': 128, 'activation': 'relu'}

Codificación real de Repetition: [8, 3, 5, 0]
Decodificación Repetition: {'type': 'Repetition', 'repetition_layers': 3, 'repetition_count': 5}


## Complete archs


In [68]:
# Clase para capas neutrales 'DontCare'
class DontCareLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(DontCareLayer, self).__init__()

    def call(self, inputs):
        return inputs

In [69]:
def encode_model_architecture(model_dict, max_alleles=48):
    """
    Codifica la arquitectura del modelo en una lista de valores reales de longitud `max_alleles`.
    """
    encoded_layers = []
    total_alleles = 0

    for layer in model_dict['layers']:
        # Codificar la capa actual
        if layer['type'] == 'Repetition':  # Codificar repetición de capas
            encoded_layer = encode_layer_params(
                layer_type_idx=8,  # índice de 'Repetition'
                param1=layer.get('repetition_layers', 0),
                param2=layer.get('repetition_count', 1)
            )
        else:
            encoded_layer = encode_layer_params(
                layer_type_idx=next(key for key, value in layer_type_options.items() if value == layer['type']),
                param1=next((key for key, value in filter_options.items() if value == layer.get('filters', 8.0)), 0),
                param2=next((key for key, value in stride_options.items() if value == layer.get('strides', 1.0)), 0),
                param3=next((key for key, value in activation_options.items() if value == layer.get('activation', 'relu')), 0)
            )
        
        # Añadir la codificación de la capa a la lista de alelos
        encoded_layers.extend(encoded_layer)
        total_alleles += len(encoded_layer)

    # Rellenar con 'DontCare' si el total de alelos es menor que `max_alleles`
    while total_alleles < max_alleles:
        dont_care_encoding = encode_layer_params(7)  # índice de 'DontCare'
        encoded_layers.extend(dont_care_encoding)
        total_alleles += len(dont_care_encoding)

    # Recortar si excede `max_alleles`
    final_encoding = encoded_layers[:max_alleles]
    print(f"Final Encoded Model: {final_encoding}")
    
    return final_encoding


In [ ]:
import random

def fixArch(encoded_model, verbose=False):
    """
    Corrige y valida la arquitectura del modelo codificado, asegurando que:
    - La primera capa es siempre Conv2D o DepthwiseConv2D (50% de probabilidad cada una).
    - La última capa es siempre Dense.
    - La capa Flatten solo aparece después del cuarto alelo.
    - No hay capas Conv2D, DepthwiseConv2D o MaxPooling después de Flatten; estas se reemplazan por DontCare.
    - Evita múltiples capas Flatten en la misma arquitectura.
    - En caso de una capa Repetition, selecciona el primer grupo válido para repetir.
    
    Parameters:
        encoded_model (list): Lista codificada de la arquitectura del modelo.
        verbose (bool): Si es True, muestra las correcciones realizadas.
    """
    
    fixed_layers = []
    input_is_flattened = False  # Indicador de si ya hubo una capa Flatten
    index = 0

    # Forzar la primera capa a ser Conv2D o DepthwiseConv2D
    first_layer_type = int(encoded_model[index])
    
    if first_layer_type not in [0, 6]:  # 0 = Conv2D, 6 = DepthwiseConv2D
        chosen_type = 0 if random.random() < 0.5 else 6
        if verbose:
            print(f"Ajustando la primera capa a {'Conv2D' if chosen_type == 0 else 'DepthwiseConv2D'}.")
        fixed_layers.extend([chosen_type, 0, 0, 0])  # Configuración básica para Conv2D o DepthwiseConv2D
        index += 4  # Saltar a la siguiente capa
    else:
        # Copiar la primera capa tal como está si es Conv2D o DepthwiseConv2D
        fixed_layers.extend(encoded_model[index:index+4])
        index += 4

    # Procesar el resto de las capas
    while index < len(encoded_model) and len(fixed_layers) < 48:
        
        layer_type = int(encoded_model[index])  # Obtener el tipo de capa actual

        # Si la capa es Repetition, mantenerla sin expandir y sin modificar
        if layer_type == 8:  # 'Repetition'
            fixed_layers.extend(encoded_model[index:index+4])
            if verbose:
                print(f"Manteniendo capa Repetition: últimas {encoded_model[index + 1]} capas, repetir {encoded_model[index + 2]} veces.")
            index += 4
            continue

        # Procesar cada tipo de capa normal con sus restricciones
        if layer_type == 0:  # 'Conv2D'
            if input_is_flattened:
                # Si ya hubo un Flatten, reemplazar Conv2D con DontCare
                if verbose:
                    print("Conv2D después de Flatten, reemplazando con DontCare.")
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                filter_idx = min(max(int(encoded_model[index + 1]), 0), 3)
                stride_idx = min(max(int(encoded_model[index + 2]), 0), 1)
                activation_idx = min(max(int(encoded_model[index + 3]), 0), 3)
                if verbose:
                    print(f"Conv2D corregido: filtros={filter_idx}, stride={stride_idx}, activación={activation_idx}")
                fixed_layers.extend([layer_type, filter_idx, stride_idx, activation_idx])
        
        elif layer_type == 6:  # 'DepthwiseConv2D'
            if input_is_flattened:
                # Si ya hubo un Flatten, reemplazar DepthwiseConv2D con DontCare
                if verbose:
                    print("DepthwiseConv2D después de Flatten, reemplazando con DontCare.")
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                filter_idx = min(max(int(encoded_model[index + 1]), 0), 3)
                stride_idx = min(max(int(encoded_model[index + 2]), 0), 1)
                activation_idx = min(max(int(encoded_model[index + 3]), 0), 3)
                if verbose:
                    print(f"DepthwiseConv2D corregido: filtros={filter_idx}, stride={stride_idx}, activación={activation_idx}")
                fixed_layers.extend([layer_type, filter_idx, stride_idx, activation_idx])
        
        elif layer_type == 2:  # 'MaxPooling'
            if input_is_flattened:
                # Si hay una MaxPooling después de un Flatten, reemplazar por DontCare
                if verbose:
                    print("MaxPooling después de Flatten, reemplazando con DontCare.")
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                stride_idx = min(max(int(encoded_model[index + 1]), 0), 1)
                if verbose:
                    print(f"MaxPooling corregido: stride={stride_idx}")
                fixed_layers.extend([layer_type, stride_idx, 0, 0])
        
        elif layer_type == 3:  # 'Dropout'
            rate_idx = min(max(int(encoded_model[index + 1]), 0), 3)
            if verbose:
                print(f"Dropout corregido: tasa={rate_idx}")
            fixed_layers.extend([layer_type, rate_idx, 0, 0])
        
        elif layer_type == 4:  # 'Dense'
            neuron_idx = min(max(int(encoded_model[index + 1]), 0), 3)
            activation_idx = min(max(int(encoded_model[index + 2]), 0), 3)
            if verbose:
                print(f"Dense corregido: neuronas={neuron_idx}, activación={activation_idx}")
            fixed_layers.extend([layer_type, neuron_idx, activation_idx, 0])
        
        elif layer_type == 1:  # 'BatchNorm'
            fixed_layers.extend([layer_type, 0, 0, 0])
            if verbose:
                print("BatchNorm: sin cambios.")
        
        elif layer_type == 5:  # 'Flatten'
            if len(fixed_layers) < 16:  # Si está antes de la cuarta capa (4x4=16 alelos)
                # Reemplazar Flatten no permitido en capas iniciales con DontCare
                if verbose:
                    print("Flatten en posición inicial no permitida, reemplazando con DontCare.")
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            elif input_is_flattened:
                # Si ya hay una capa Flatten en el modelo, reemplazar esta con DontCare
                if verbose:
                    print("Flatten adicional encontrado, reemplazando con DontCare.")
                fixed_layers.extend([7, 0, 0, 0])  # DontCare
            else:
                fixed_layers.extend([layer_type, 0, 0, 0])
                input_is_flattened = True
                if verbose:
                    print("Flatten: añadido correctamente.")
        
        elif layer_type == 7:  # 'DontCare'
            fixed_layers.extend([layer_type, 0, 0, 0])
            if verbose:
                print("DontCare: sin cambios.")

        index += 4  # Pasar al siguiente grupo de alelos

    # Asegurarse de que la última capa sea Dense
    if fixed_layers[-4] != 4:  # Si la última capa no es Dense
        if verbose:
            print("Ajustando la última capa a Dense con activación sigmoid.")
        fixed_layers[-4:] = [4, 0, 2, 0]  # Cambiar a capa Dense con activación 'sigmoid'

    # Truncar a 48 alelos si excede el límite
    return fixed_layers[:48]


In [71]:
# Ejecutar ejemplos de prueba para la función fixArch con arquitecturas de 12 capas (48 valores en total)

# Ejemplo 1: Arquitectura sin Flatten antes de Dense y con capas adicionales
encoded_model_1 = [
    0, 3, 0, 0,    # Conv2D, 32 filtros, stride 1, activación relu
    2, 1, 0, 0,    # MaxPooling, stride 2
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu (sin Flatten antes)
    7, 0, 0, 0,    # DontCare
    1, 0, 0, 0,    # BatchNorm
    3, 2, 0, 0,    # Dropout, tasa 0.4
    0, 1, 1, 0,    # Conv2D, 16 filtros, stride 2, activación relu
    6, 1, 1, 0,    # DepthwiseConv2D, 16 filtros, stride 2, activación relu
    5, 0, 0, 0,    # Flatten
    4, 3, 2, 0,    # Dense, 256 neuronas, activación sigmoid
    4, 1, 3, 0,    # Dense, 32 neuronas, activación tanh
    4, 1, 2, 0     # Dense, 32 neuronas, activación sigmoid
]

# Ejemplo 2: Arquitectura con Flatten al inicio (inválido) y sin Dense al final
encoded_model_2 = [
    5, 0, 0, 0,    # Flatten (inválido al inicio)
    0, 2, 1, 1,    # Conv2D, 30 filtros, stride 2, activación leaky_relu
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    1, 0, 0, 0,    # BatchNorm
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    3, 0, 0, 0,    # Dropout, tasa 0.2
    1, 0, 0, 0,    # BatchNorm
    4, 3, 3, 0,    # Dense, 256 neuronas, activación tanh
    7, 0, 0, 0     # DontCare
]

# Ejemplo 3: Arquitectura con Repetition y sin Dense al final
encoded_model_3 = [
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    8, 2, 2, 0,    # Repetition, repite las últimas 2 capas 2 veces
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    0, 3, 0, 1,    # Conv2D, 32 filtros, stride 1, activación leaky_relu
    1, 0, 0, 0,    # BatchNorm
    5, 0, 0, 0,    # Flatten
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    3, 2, 0, 0     # Dropout, tasa 0.4
]

# Ejemplo 4: Arquitectura con parámetros fuera de límite
encoded_model_4 = [
    0, 4, 2, 1,    # Conv2D, 64 filtros (fuera de límite), stride 2, activación leaky_relu
    3, 5, 0, 0,    # Dropout, tasa 0.6 (fuera de límite)
    4, 5, 3, 0,    # Dense, 512 neuronas (fuera de límite), activación sigmoid
    6, 4, 2, 0,    # DepthwiseConv2D, 64 filtros (fuera de límite), stride 2, activación relu
    1, 0, 0, 0,    # BatchNorm
    3, 3, 0, 0,    # Dropout, tasa 0.5
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    0, 1, 0, 0,    # Conv2D, 16 filtros, stride 1, activación relu
    4, 3, 1, 0,    # Dense, 256 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    4, 1, 2, 0     # Dense, 32 neuronas, activación sigmoid
]
# Ejemplo 5: Arquitectura con capa de Repetition que repite las últimas capas
encoded_model_5 = [
    0, 2, 1, 0,    # Conv2D, 30 filtros, stride 2, activación relu
    8, 2, 3, 0,    # Repetition, repite las últimas 2 capas 3 veces
    5, 0, 0, 0,    # Flatten
    2, 1, 0, 0,    # MaxPooling, stride 2
    3, 1, 0, 0,    # Dropout, tasa 0.3
    6, 2, 1, 0,    # DepthwiseConv2D, 30 filtros, stride 2, activación relu
    0, 3, 0, 1,    # Conv2D, 32 filtros, stride 1, activación leaky_relu
    1, 0, 0, 0,    # BatchNorm
    5, 0, 0, 0,    # Flatten
    4, 2, 1, 0,    # Dense, 128 neuronas, activación leaky_relu
    7, 0, 0, 0,    # DontCare
    3, 2, 0, 0     # Dropout, tasa 0.4
]

# Lista de modelos para probar
model_examples = [encoded_model_1, encoded_model_2, encoded_model_3, encoded_model_4, encoded_model_5]

# Ejecución de pruebas
for i, model in enumerate(model_examples, 1):
    print(f"\n--- Ejemplo {i} ---")
    fixed_model = fixArch(model, verbose=True)
    print(f"Modelo Original: {len(model)} alelos\n{model}") 
    print(f"Modelo Corregido: {len(fixed_model)} alelos\n{fixed_model}")



--- Ejemplo 1 ---
MaxPooling corregido: stride=1
Dense corregido: neuronas=2, activación=1
DontCare: sin cambios.
BatchNorm: sin cambios.
Dropout corregido: tasa=2
Conv2D corregido: filtros=1, stride=1, activación=0
DepthwiseConv2D corregido: filtros=1, stride=1, activación=0
Flatten: añadido correctamente.
Dense corregido: neuronas=3, activación=2
Dense corregido: neuronas=1, activación=3
Dense corregido: neuronas=1, activación=2
Modelo Original: 48 alelos
[0, 3, 0, 0, 2, 1, 0, 0, 4, 2, 1, 0, 7, 0, 0, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 1, 1, 0, 6, 1, 1, 0, 5, 0, 0, 0, 4, 3, 2, 0, 4, 1, 3, 0, 4, 1, 2, 0]
Modelo Corregido: 48 alelos
[0, 3, 0, 0, 2, 1, 0, 0, 4, 2, 1, 0, 7, 0, 0, 0, 1, 0, 0, 0, 3, 2, 0, 0, 0, 1, 1, 0, 6, 1, 1, 0, 5, 0, 0, 0, 4, 3, 2, 0, 4, 1, 3, 0, 4, 1, 2, 0]

--- Ejemplo 2 ---
Ajustando la primera capa a DepthwiseConv2D.
Conv2D corregido: filtros=2, stride=1, activación=1
MaxPooling corregido: stride=1
Dropout corregido: tasa=1
BatchNorm: sin cambios.
Conv2D corregido: filt

In [72]:
def decode_model_architecture(encoded_model):
    """
    Decodifica la arquitectura del modelo a partir de una lista codificada de valores (índices).
    Expande las capas de repetición y asegura una capa Flatten antes de la última capa Dense.
    """
    model_dict = {'layers': []}
    index = 0

    # Decodificar cada capa en grupos de 4 valores
    while index < len(encoded_model):
        layer_type = int(encoded_model[index])  # Tipo de capa
        param1 = int(encoded_model[index + 1])  # Primer parámetro (ej., filtros o capas de repetición)
        param2 = int(encoded_model[index + 2])  # Segundo parámetro (ej., stride o número de repeticiones)
        param3 = int(encoded_model[index + 3])  # Tercer parámetro (ej., activación o tasa de dropout)

        if layer_type == 8:  # 'Repetition'
            # Decodificar capa de repetición
            repetition_layers = param1
            repetition_count = param2
            layers_to_repeat = model_dict['layers'][-repetition_layers:]  # Tomar las últimas `repetition_layers` capas
            if len(layers_to_repeat) > 0:  # Solo repetir si hay capas seleccionadas
                for _ in range(repetition_count):
                    model_dict['layers'].extend(layers_to_repeat)
        else:
            # Decodificar capa normal
            decoded_layer = {}

            if layer_type == 0:  # 'Conv2D'
                decoded_layer = {
                    'type': 'Conv2D',
                    'filters': filter_options.get(param1, 8),
                    'strides': stride_options.get(param2, 1),
                    'activation': activation_options.get(param3, 'relu')
                }
            elif layer_type == 6:  # 'DepthwiseConv2D'
                decoded_layer = {
                    'type': 'DepthwiseConv2D',
                    'filters': filter_options.get(param1, 8),
                    'strides': stride_options.get(param2, 1),
                    'activation': activation_options.get(param3, 'relu')
                }
            elif layer_type == 2:  # 'MaxPooling'
                decoded_layer = {
                    'type': 'MaxPooling',
                    'strides': stride_options.get(param1, 1)
                }
            elif layer_type == 3:  # 'Dropout'
                decoded_layer = {
                    'type': 'Dropout',
                    'rate': dropout_options.get(param1, 0.2)
                }
            elif layer_type == 4:  # 'Dense'
                decoded_layer = {
                    'type': 'Dense',
                    'units': neuron_options.get(param1, 1),
                    'activation': activation_options.get(param2, 'relu')
                }
            elif layer_type == 1:  # 'BatchNorm'
                decoded_layer = {'type': 'BatchNorm'}
            elif layer_type == 5:  # 'Flatten'
                decoded_layer = {'type': 'Flatten'}
            elif layer_type == 7:  # 'DontCare'
                decoded_layer = {'type': 'DontCare'}

            # Añadir la capa decodificada al modelo
            model_dict['layers'].append(decoded_layer)

        index += 4  # Pasar al siguiente grupo de alelos

    # Insertar una capa Flatten antes de la última capa Dense
    if len(model_dict['layers']) > 1 and model_dict['layers'][-1]['type'] == 'Dense':
        model_dict['layers'].insert(-1, {'type': 'Flatten'})

    # Asegurarse de que la última capa es 'Dense' (si no lo es, forzamos una capa 'Dense')
    if model_dict['layers'][-1]['type'] != 'Dense':
        model_dict['layers'].append({'type': 'Dense', 'units': 1, 'activation': 'sigmoid'})

    return model_dict


In [73]:
# Reparar cada ejemplo con fixArch y luego decodificar con decode_model_architecture
repaired_and_decoded_models = []

for i, encoded_model in enumerate([encoded_model_1, encoded_model_2, encoded_model_3, encoded_model_4, encoded_model_5], 1):
    # Reparar el modelo
    repaired_model = fixArch(encoded_model, verbose=True)
    # Decodificar el modelo reparado
    decoded_model = decode_model_architecture(repaired_model)
    # Guardar el modelo decodificado
    repaired_and_decoded_models.append(decoded_model)
    print(f"\n--- Modelo {i} Decodificado ---")
    print(decoded_model)

# Opcional: Mostrar todos los modelos decodificados juntos
print("\n--- Todos los Modelos Decodificados ---")
for i, model in enumerate(repaired_and_decoded_models, 1):
    print(f"Modelo {i} Decodificado:", model)

MaxPooling corregido: stride=1
Dense corregido: neuronas=2, activación=1
DontCare: sin cambios.
BatchNorm: sin cambios.
Dropout corregido: tasa=2
Conv2D corregido: filtros=1, stride=1, activación=0
DepthwiseConv2D corregido: filtros=1, stride=1, activación=0
Flatten: añadido correctamente.
Dense corregido: neuronas=3, activación=2
Dense corregido: neuronas=1, activación=3
Dense corregido: neuronas=1, activación=2

--- Modelo 1 Decodificado ---
{'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Dense', 'units': 128, 'activation': 'leaky_relu'}, {'type': 'DontCare'}, {'type': 'BatchNorm'}, {'type': 'Dropout', 'rate': 0.4}, {'type': 'Conv2D', 'filters': 16, 'strides': 2, 'activation': 'relu'}, {'type': 'DepthwiseConv2D', 'filters': 16, 'strides': 2, 'activation': 'relu'}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 256, 'activation': 'sigmoid'}, {'type': 'Dense', 'units': 32, 'activation': 'tanh'}, {'type'

In [74]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, DepthwiseConv2D



def build_tf_model_from_dict(model_dict, input_shape=(28, 28, 3)):
    """
    Construye un modelo de TensorFlow a partir de un diccionario JSON expandido.
    """
    print("\nConstruyendo el modelo en TensorFlow desde el JSON expandido...")
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=input_shape))

    for layer in model_dict['layers']:
        if layer['type'] == 'Conv2D':
            model.add(Conv2D(filters=layer['filters'], kernel_size=(3, 3), strides=int(layer['strides']), padding='same', activation=layer['activation']))
        
        elif layer['type'] == 'DepthwiseConv2D':
            model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=int(layer['strides']), padding='same', activation=layer['activation']))
        
        elif layer['type'] == 'BatchNorm':
            model.add(BatchNormalization())
        
        elif layer['type'] == 'MaxPooling':
            model.add(MaxPooling2D(pool_size=(2, 2), strides=int(layer['strides']), padding='same'))
        
        elif layer['type'] == 'Flatten':
            model.add(Flatten())
        
        elif layer['type'] == 'Dense':
            model.add(Dense(units=int(layer['units']), activation=layer['activation']))
        
        elif layer['type'] == 'Dropout':
            model.add(Dropout(rate=layer['rate']))
        
        elif layer['type'] == 'DontCare':
            model.add(DontCareLayer())
    
    return model


In [75]:
# Ejemplo 1: build_CNN_LF_model
model_CNN_LF = {
    "layers": [
        {"type": "Conv2D", "filters": 30, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},  # Revisar 'filters'
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.3},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 2: build_reduced_model
model_reduced = {
    "layers": [
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "leaky_relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 3: build_Spectro_CNN_model
model_spectro_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "MaxPooling", "strides": 2}, 
         
         
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
    {"type":"Flatten"},
    {"type":"Dense","units":256,"activation":"relu"},   
    {"type":"Dropout","rate":0.5},
                
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo corregido: build_Spectro_CNN_model con capas de repetición
model_spectro_CNN_with_repetition = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2}, 
        # Aquí indicamos que las siguientes 10 capas (5 Conv2D + 5 BatchNorm) se repiten 10 veces
        {"type": "Repetition", "repetition_layers": 3, "repetition_count": 31},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


# Ejemplo 4: Simple Conv2D model
model_simple_conv = {
    "layers": [
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 5: Simple Dense model
model_dense_only = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 256, "activation": "sigmoid"}
    ]
}

# Ejemplo 6: Small CNN model with Dropout
model_small_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 7: Deep CNN model
model_deep_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 8: Basic Dense with Dropout
model_dense_dropout = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 32, "activation": "tanh"}
    ]
}

model_with_depthwise = {
    "layers": [
        {"type": "DepthwiseConv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Dense", "units": 128, "activation": "sigmoid"},
        {"type": "Dense", "units": 1, "activation": "tanh"}
    ]
}



In [76]:
def process_model_pipeline(model_dict, input_shape=(28, 28, 3), verbose=False):
    """
    Procesa el pipeline de creación del modelo desde el JSON inicial hasta el modelo de TensorFlow.
    
    Parameters:
        model_dict (dict): Modelo en formato JSON.
        input_shape (tuple): Forma de entrada para el modelo.
        verbose (bool): Si es True, muestra los detalles de cada etapa del proceso.
        
    Returns:
        tf.keras.Model: El modelo de TensorFlow construido a partir del JSON procesado.
    """
    # Codificar el modelo
    encoded_model = encode_model_architecture(model_dict, max_alleles=48)
    if verbose:
        print(f"\nModelo codificado: {encoded_model}")

    # Reparar el modelo codificado
    fixed_model = fixArch(encoded_model, verbose=verbose)
    if verbose:
        print(f"\nModelo codificado y reparado: {fixed_model}")

    # Decodificar el modelo reparado
    decoded_model = decode_model_architecture(fixed_model)
    if verbose:
        print(f"\nModelo decodificado:\n{decoded_model}")

    # Construir el modelo de TensorFlow
    tf_model = build_tf_model_from_dict(decoded_model, input_shape=input_shape)
    if verbose:
        print("\nModelo de TensorFlow construido.")
        tf_model.summary()
    
    return tf_model


# Ejemplo de uso con un modelo y verbose activado



In [77]:

# Verificación de los modelos
print("Verifications:")

print("\nModel 1: CNN_LF")
tf_model_example = process_model_pipeline(model_CNN_LF, verbose=True)  # Ejemplo 1

print("\nModel 2: Reduced")
tf_model_example = process_model_pipeline(model_reduced, verbose=True)  # Ejemplo 2

print("\nModel 3: Spectro CNN")
tf_model_example = process_model_pipeline(model_spectro_CNN, verbose=True)  # Ejemplo 3

print("\nModel 3.5 Spectro CNN with repetition")
tf_model_example = process_model_pipeline(model_spectro_CNN_with_repetition, verbose=True)  # Ejemplo 3

print("\nModel 4: Simple Conv2D")
tf_model_example = process_model_pipeline(model_simple_conv, verbose=True)  # Ejemplo 4

print("\nModel 5: Dense Only")
tf_model_example = process_model_pipeline(model_dense_only, verbose=True)  # Ejemplo 5

print("\nModel 6: Small CNN")
tf_model_example = process_model_pipeline(model_small_CNN, verbose=True)  # Ejemplo 6

print("\nModel 7: Deep CNN")
tf_model_example = process_model_pipeline(model_deep_CNN, verbose=True)  # Ejemplo 7

print("\nModel 8: Dense with Dropout")
tf_model_example = process_model_pipeline(model_dense_dropout, verbose=True)  # Ejemplo 8

print("\nModel with DepthwiseConv2D")
tf_model_example = process_model_pipeline(model_with_depthwise)



Verifications:

Model 1: CNN_LF
Final Encoded Model: [0, 2, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 5, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 2]

Modelo codificado: [0, 2, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 5, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 2]
Dropout corregido: tasa=0
BatchNorm: sin cambios.
MaxPooling corregido: stride=0
Conv2D corregido: filtros=1, stride=0, activación=0
Dropout corregido: tasa=0
BatchNorm: sin cambios.
MaxPooling corregido: stride=0
Flatten: añadido correctamente.
Dense corregido: neuronas=0, activación=0
Dropout corregido: tasa=0
Dense corregido: neuronas=0, activación=0

Modelo codificado y reparado: [0, 2, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 0]

Modelo decodificado:
{'layers': [{'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'}, 

# Testing random generated architectures


In [78]:
import random

def generate_random_architecture():
    """
    Genera una arquitectura aleatoria con una probabilidad del 25% de que sea inválida.
    
    Returns:
        dict: Arquitectura aleatoria en formato JSON.
    """
    num_layers = 12  # Número aleatorio de capas entre 5 y 15
    layers = []
    
    # 25% de probabilidad de que sea una arquitectura no válida
    invalid_architecture = random.random() < 0.25
    
    # Generar capa inicial (puede ser válida o no válida)
    if invalid_architecture:
        layer_type = random.choice(['Dense', 'BatchNorm', 'Dropout'])  # Capa no válida para iniciar
    else:
        layer_type = random.choice(['Conv2D', 'DepthwiseConv2D'])  # Capa válida
    layers.append(generate_layer(layer_type))

    # Generar el resto de las capas aleatoriamente
    for _ in range(1, num_layers - 1):
        layer_type = random.choice([
            'Conv2D', 'DepthwiseConv2D', 'BatchNorm', 'MaxPooling', 'Dropout', 
            'Dense', 'Flatten', 'DontCare', 'Repetition'
        ])
        layers.append(generate_layer(layer_type))

    # Añadir última capa (puede ser válida o no válida)
    if invalid_architecture:
        layers.append(generate_layer(random.choice(['Conv2D', 'Dropout', 'Flatten'])))  # Última capa no válida
    else:
        layers.append(generate_layer('Dense', force_valid=True))  # Última capa válida

    return {"layers": layers}

def generate_layer(layer_type, force_valid=False):
    """
    Genera una capa aleatoria según el tipo.
    
    Parameters:
        layer_type (str): Tipo de la capa (e.g., 'Conv2D', 'Dense').
        force_valid (bool): Si es True, se generan parámetros válidos forzados.
        
    Returns:
        dict: Diccionario con los parámetros de la capa generada.
    """
    if layer_type == 'Conv2D':
        return {
            "type": "Conv2D",
            "filters": random.choice([8, 16, 30, 32]),
            "strides": random.choice([1, 2]),
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"])
        }
    elif layer_type == 'DepthwiseConv2D':
        return {
            "type": "DepthwiseConv2D",
            "filters": random.choice([8, 16, 30, 32]),
            "strides": random.choice([1, 2]),
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"])
        }
    elif layer_type == 'BatchNorm':
        return {"type": "BatchNorm"}
    elif layer_type == 'MaxPooling':
        return {
            "type": "MaxPooling",
            "strides": random.choice([1, 2])
        }
    elif layer_type == 'Dropout':
        return {
            "type": "Dropout",
            "rate": random.choice([0.2, 0.3, 0.4, 0.5])
        }
    elif layer_type == 'Dense':
        return {
            "type": "Dense",
            "units": random.choice([1, 32, 128, 256]),
            "activation": random.choice(["relu", "leaky_relu", "sigmoid", "tanh"]) if not force_valid else "sigmoid"
        }
    elif layer_type == 'Flatten':
        return {"type": "Flatten"}
    elif layer_type == 'DontCare':
        return {"type": "DontCare"}
    elif layer_type == 'Repetition':
        return {
            "type": "Repetition",
            "repetition_layers": random.randint(1, 5),  # Número de capas a repetir
            "repetition_count": random.randint(1, 5)   # Número de repeticiones
        }
    return {}

def generate_and_print_tf_models(num_models=5):
    """
    Genera, repara y construye múltiples modelos aleatorios de TensorFlow.
    
    Parameters:
        num_models (int): Número de modelos a generar y procesar.
    """
    for i in range(num_models):
        print(f"\n--- Modelo Aleatorio {i+1} ---")
        
        # Generar una arquitectura aleatoria
        random_architecture = generate_random_architecture()
        print(f"\nArquitectura Generada: {random_architecture}")
        
        # Procesar la arquitectura generada a través del pipeline
        tf_model = process_model_pipeline(random_architecture, verbose=True)
        print("\n")

# Ejemplo de ejecución
generate_and_print_tf_models(num_models=500)



--- Modelo Aleatorio 1 ---

Arquitectura Generada: {'layers': [{'type': 'Dropout', 'rate': 0.3}, {'type': 'Dense', 'units': 1, 'activation': 'relu'}, {'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.3}, {'type': 'Flatten'}, {'type': 'Dropout', 'rate': 0.5}, {'type': 'DepthwiseConv2D', 'filters': 30, 'strides': 1, 'activation': 'sigmoid'}, {'type': 'Conv2D', 'filters': 8, 'strides': 2, 'activation': 'leaky_relu'}, {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'leaky_relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'Dropout', 'rate': 0.3}, {'type': 'Dropout', 'rate': 0.2}]}
Final Encoded Model: [3, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 6, 2, 0, 2, 0, 0, 1, 1, 0, 3, 0, 1, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0]

Modelo codificado: [3, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 6, 2, 0, 2, 0, 0, 1, 1, 0, 3, 0, 1, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0]
Ajustando la primer

ValueError: Input 0 of layer "conv2d_546" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 3136)